# Spatial normalization of fluorescence to account for differences in illumination brightness

In [ ]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib as mpl
import time
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as patches
from scipy import stats
from scipy.spatial import distance
import numpy as np
from scipy import interpolate
from pathlib import Path
from sklearn.linear_model import LinearRegression

from fluorescence_processing.fluor_util import ScatterSelectorGating
from bsccm import BSCCM

bsccm = BSCCM(str(Path.home()) + '/BSCCM_local/BSCCM/')


def lowess_2d(y_pos, x_pos, z_val, yx_query_points,
              alpha=0.1, weight_fn='tricubic'):
   
    distances = np.linalg.norm(yx_query_points[:, None] - 
                       np.stack([y_pos, x_pos], axis=1)[None], axis=2)

    # find set of closest points to each query point
    closest_indices = np.argsort(distances, axis=1)[..., :int(alpha * y_pos.size)]
    closest_coords = np.stack([y_pos[closest_indices], x_pos[closest_indices]], axis=2)
    closest_targets = z_val[closest_indices]

    # Compute weighting function based on distance from center of window

    weight_distances = np.sqrt(np.sum((closest_coords - yx_query_points[:, None]) ** 2, axis=2))
    #normalize to 1
    weight_distances /= np.max(weight_distances)
    if weight_fn == 'tricubic':
        weights = (1 - weight_distances ** 3) ** 3
    elif weight_fn == 'gaussian':
        weights = np.exp( -0.5*(weight_distances / window_sigma) ** 2 )
    else:
        raise Exception('unknown weight fn')
    weights = np.sqrt(weights)

    #solve least squares problems and make predicitions
    predictions = []
    for query_point, w, X, y in zip(
        yx_query_points, weights, closest_coords, closest_targets):
        reg = LinearRegression().fit(X, y, w)
        predictions.append(reg.predict(query_point[None]))
    return np.array(predictions)



def plot_gridded_lowess(y_pos, x_pos, fluor, yx_query_points, ax, N = 20):
    #do LOWESS on a grid
    yy, xx = np.meshgrid(np.linspace(0, 2056, N), np.linspace(0, 2056, N))
    yx_query_points = np.stack([yy, xx], axis=-1).reshape([-1, 2])

    # compute alpha adaptively so it corresponds to same number cells
    alpha = 2500 / x_pos.shape[0] 
    predictions = lowess_2d(y_pos, x_pos, fluor, 
              yx_query_points, alpha=alpha, weight_fn='tricubic')
    predictions = predictions.reshape([N, N])

    im = ax.imshow(predictions, cmap='inferno')
    plt.colorbar(im, ax=ax)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.set_yticks([])
    ax.set_xticks([])

In [2]:
bsccm.surface_marker_dataframe

,global_index,Fluor_690-_total_raw,Fluor_690-_background,Fluor_627-673_total_raw,Fluor_627-673_background,Fluor_585-625_total_raw,Fluor_585-625_background,Fluor_550-570_total_raw,Fluor_550-570_background,Fluor_500-550_total_raw,Fluor_500-550_background,Fluor_426-446_total_raw,Fluor_426-446_background
0,0,1159046.0,263865.0,1428465.0,325809.0,1180954.0,274251.0,783328.0,205790.0,1811893.0,422595.0,893586.0,155803.0
1,1,1056675.0,266847.0,1342093.0,334576.0,1134226.0,280610.0,801460.0,212972.0,1761581.0,428277.0,733878.0,154204.0
2,2,1038178.0,261834.0,1328895.0,325528.0,1124847.0,272854.0,769109.0,203352.0,1754821.0,419895.0,729227.0,153341.0
3,3,1015439.0,263122.0,1192476.0,295889.0,1010930.0,245912.0,595072.0,156292.0,1552457.0,374031.0,663304.0,147709.0
4,4,1061897.0,274378.0,1269530.0,314392.0,1066572.0,259812.0,680883.0,179596.0,1657599.0,398542.0,667175.0,150189.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
412936,412936,956817.0,258138.0,967237.0,247002.0,817377.0,203141.0,665249.0,177675.0,1264239.0,309816.0,486165.0,127090.0
412937,412937,969882.0,260998.0,982940.0,251954.0,829561.0,206201.0,727921.0,194985.0,1284115.0,319217.0,474215.0,126315.0
412938,412938,937043.0,253535.0,931074.0,240860.0,782612.0,198369.0,662844.0,177569.0,1197791.0,302388.0,475595.0,126000.0
412939,412939,908121.0,244698.0,1007356.0,256514.0,790513.0,194905.0,727145.0,194282.0,1290193.0,317242.0,486935.0,126193.0


## Plan
 - Visualize current state
 - Compute median of background measurements to get slide specific, channel_specific background
 - Compute median of cells to get slide specific, channel_specific, foreground
 - Compute Loess of each one to get a continuous measurement
 - Subtract background from foregound and normalize to get shading correction
 - final result: foreground minus background, divide by normalized shading, visualize result


## Visualize spatial histograms of raw brightness

In [3]:
def show_spatial_histograms(channel_names):
    replicate = 0
    antibodies_list = bsccm.index_dataframe['antibodies'].unique()

    for batch in range(2):
        fig, ax = plt.subplots(len(antibodies_list), len(channel_names), figsize=(9,7))
        plt.title('Batch {}'.format(batch))
        for i, antibodies in enumerate(antibodies_list):
            mask = np.logical_and(np.logical_and(bsccm.index_dataframe['antibodies'] == antibodies,
                                                bsccm.index_dataframe['batch'] == batch), 
                                                bsccm.index_dataframe['slide_replicate'] == replicate)
            mask_indices = np.flatnonzero(mask)

            for j, channel in enumerate(channel_names):
                if j == 0:
                    ax[i, j].set_ylabel(antibodies)
                y_pos = bsccm.index_dataframe.loc[mask_indices, 'position_in_fov_y_pix'].to_numpy()
                x_pos = bsccm.index_dataframe.loc[mask_indices, 'position_in_fov_x_pix'].to_numpy()
                fluor = bsccm.surface_marker_dataframe.loc[mask_indices, channel].to_numpy()
                stat = stats.binned_statistic_2d(y_pos, x_pos, fluor, bins=10, 
                                                 statistic=lambda x: np.percentile(x, 50)).statistic

                contrast_max = np.nanpercentile(stat, 95)
                im = ax[i, j].imshow(stat, cmap='inferno', vmax=contrast_max)
                plt.colorbar(im, ax=ax[i, j])
                ax[i, j].set_yticklabels([])
                ax[i, j].set_xticklabels([])
                ax[i, j].set_yticks([])
                ax[i, j].set_xticks([])
        fig.suptitle('Batch {}'.format(batch))


In [4]:
%matplotlib widget

import matplotlib.pyplot as plt

channel_names = [
    'Fluor_426-446_total_raw',
    'Fluor_500-550_total_raw',
    'Fluor_550-570_total_raw',
    'Fluor_585-625_total_raw',
    'Fluor_627-673_total_raw',
    'Fluor_690-_total_raw',
    ]

show_spatial_histograms(channel_names)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Visualize spatial histograms of backgrounds

In [5]:
background_names = ['Fluor_426-446_background',
 'Fluor_500-550_background',
 'Fluor_550-570_background',
 'Fluor_585-625_background',
 'Fluor_627-673_background',
 'Fluor_690-_background']

show_spatial_histograms(background_names)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Visualize computed smooth function of background intensity pattern

In [6]:
replicate = 0
antibodies_list = bsccm.index_dataframe['antibodies'].unique()

background_names = ['Fluor_426-446_background',
 'Fluor_500-550_background',
 'Fluor_550-570_background',
 'Fluor_585-625_background',
 'Fluor_627-673_background',
 'Fluor_690-_background']

for batch in range(2):
    fig, ax = plt.subplots(len(antibodies_list), len(background_names), figsize=(8,10))
    plt.title('Batch {}'.format(batch))
    for i, antibodies in enumerate(antibodies_list):
        print('{} {}\r'.format(batch, antibodies))
        mask = np.logical_and(np.logical_and(bsccm.index_dataframe['antibodies'] == antibodies,
                                            bsccm.index_dataframe['batch'] == batch), 
                                            bsccm.index_dataframe['slide_replicate'] == replicate)
        mask_indices = np.flatnonzero(mask)

        for j, channel in enumerate(background_names):
            if j == 0:
                ax[i, j].set_ylabel(antibodies)
            y_pos = bsccm.index_dataframe.loc[mask_indices, 'position_in_fov_y_pix'].to_numpy()
            x_pos = bsccm.index_dataframe.loc[mask_indices, 'position_in_fov_x_pix'].to_numpy()
            fluor = bsccm.surface_marker_dataframe.loc[mask_indices, background_names[j]].to_numpy()
            
            plot_gridded_lowess(y_pos, x_pos, fluor, yx_query_points, ax[i, j])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0 CD45
0 CD123
0 unstained
0 CD19
0 CD56
0 all
0 CD14
0 CD16
0 HLA-DR
0 CD3


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1 CD45
1 CD123
1 unstained
1 CD19
1 CD56
1 all
1 CD14
1 CD16
1 HLA-DR
1 CD3


### 1) Go through all slides, compute the background for each, and store lowess background subtracted values

In [ ]:
background_names = ['Fluor_426-446_background',
 'Fluor_500-550_background',
 'Fluor_550-570_background',
 'Fluor_585-625_background',
 'Fluor_627-673_background',
 'Fluor_690-_background']


antibodies_list = bsccm.index_dataframe['antibodies'].unique()

for batch in range(2):
    for i, antibodies in enumerate(antibodies_list):
        print(antibodies)
        for replicate in range(2):
            mask = np.logical_and(np.logical_and(bsccm.index_dataframe['antibodies'] == antibodies,
                                            bsccm.index_dataframe['batch'] == batch), 
                                            bsccm.index_dataframe['slide_replicate'] == replicate)
            mask_indices = np.flatnonzero(mask)
            if mask_indices.size == 0:
                continue

            for j, channel in enumerate(background_names):            
                print(channel)
                y_pos = bsccm.index_dataframe.loc[mask_indices, 'position_in_fov_y_pix'].to_numpy()
                x_pos = bsccm.index_dataframe.loc[mask_indices, 'position_in_fov_x_pix'].to_numpy()
                fluor = bsccm.surface_marker_dataframe.loc[mask_indices, channel].to_numpy()


                yx_query_points = np.stack([y_pos, x_pos], axis=-1)

                # compute alpha adaptively so it corresponds to same number cells
                alpha = 2500 / x_pos.shape[0] 
                predictions = lowess_2d(y_pos, x_pos, fluor, 
                          yx_query_points, alpha=alpha, weight_fn='tricubic')
                bsccm.surface_marker_dataframe.loc[mask_indices, channel + '_lowess'] = predictions
        
            

CD45
Fluor_426-446_background
Fluor_500-550_background
Fluor_550-570_background
Fluor_585-625_background
Fluor_627-673_background
Fluor_690-_background
CD123
Fluor_426-446_background
Fluor_500-550_background
Fluor_550-570_background
Fluor_585-625_background
Fluor_627-673_background
Fluor_690-_background
unstained
Fluor_426-446_background
Fluor_500-550_background
Fluor_550-570_background
Fluor_585-625_background
Fluor_627-673_background
Fluor_690-_background
CD19
Fluor_426-446_background
Fluor_500-550_background
Fluor_550-570_background
Fluor_585-625_background
Fluor_627-673_background
Fluor_690-_background
CD56
Fluor_426-446_background
Fluor_500-550_background
Fluor_550-570_background
Fluor_585-625_background
Fluor_627-673_background
Fluor_690-_background
all
Fluor_426-446_background
Fluor_500-550_background
Fluor_550-570_background
Fluor_585-625_background
Fluor_627-673_background
Fluor_690-_background
CD14
Fluor_426-446_background
Fluor_500-550_background
Fluor_550-570_background
Flu

In [ ]:
#Resave

home = str(Path.home())
data_root = home + '/BSCCM_local/BSCCM/'
fluor_dataframe.to_csv(data_root + 'BSCCM_surface_markers.csv', index=False)

### 2) Compute final normalized fluorescence

Pool over markers (subset of 1000 from each slide?), and compute LOWESS over background-subtracted cells divided by mean intensity

    a) Visualize the computed LOWESS
    b) Use it to compute final normalized fluorescence
    
Output: normalized fluorescence


In [ ]:
"""
Take subsets of 1000 of each slide, do slide-specific background subtraction, store results
"""

background_names_lowess = ['Fluor_426-446_background_lowess',
 'Fluor_500-550_background_lowess',
 'Fluor_550-570_background_lowess',
 'Fluor_585-625_background_lowess',
 'Fluor_627-673_background_lowess',
 'Fluor_690-_background_lowess']

channel_names = [
    'Fluor_426-446_total_raw',
    'Fluor_500-550_total_raw',
    'Fluor_550-570_total_raw',
    'Fluor_585-625_total_raw',
    'Fluor_627-673_total_raw',
    'Fluor_690-_total_raw',
    ]

antibodies_list = bsccm.index_dataframe['antibodies'].unique()

data_by_channel = {c: {'x_pos': [],
                                 'y_pos': [],
                                 'background_sub_fluor': []} for c in channel_names}

for batch in range(2):
    for i, antibodies in enumerate(antibodies_list):
        print(antibodies)
        for replicate in range(2):
            mask = np.logical_and(np.logical_and(bsccm.index_dataframe['antibodies'] == antibodies,
                                            bsccm.index_dataframe['batch'] == batch), 
                                            bsccm.index_dataframe['slide_replicate'] == replicate)
            mask_indices = np.flatnonzero(mask)
            if mask_indices.size == 0:
                continue
                
            selected_indices = np.random.choice(mask_indices, 1000)

            for ch_index in range(len(background_names_lowess)):            
                y_pos = bsccm.index_dataframe.loc[selected_indices, 'position_in_fov_y_pix'].to_numpy()
                x_pos = bsccm.index_dataframe.loc[selected_indices, 'position_in_fov_x_pix'].to_numpy()
                fluor_foreground = bsccm.surface_marker_dataframe.loc[selected_indices, 
                                                           channel_names[j]].to_numpy()
                fluor_background_lowess = bsccm.surface_marker_dataframe.loc[selected_indices, 
                                                           background_names_lowess[j]].to_numpy()
                data_by_channel[channel_names[j]]['x_pos'].append(x_pos)
                data_by_channel[channel_names[j]]['y_pos'].append(y_pos)
                data_by_channel[channel_names[j]]['background_sub_fluor'].append(y_pos) = \
                         fluor_foreground - fluor_background_lowess     

for channel in channel_names:
    data_by_channel[channel]['x_pos'] = np.concatenate(data_by_channel[channel]['x_pos'])
    data_by_channel[channel]['y_pos'] = np.concatenate(data_by_channel[channel]['y_pos'])
    data_by_channel[channel]['background_sub_fluor'] = np.concatenate(data_by_channel[channel]['background_sub_fluor'])



### Visualize LOWESS

In [ ]:
#Normalize data before computing LOWESS
# fig, ax = plt.subplots(1, 6, figsize=(8,2))

for c_index, channel in enumerate(channel_names):
    y_pos = data_by_channel[channel]['y_pos']
    x_pos = data_by_channel[channel]['x_pos']
    
    fluor = data_by_channel[channel]['background_sub_fluor']
    
    
    #TODO: how to normalize fluor -- visualize histograms
    # 1) subtract mean, divide by SD, exponentiate?

    plt.figure()
    plt.hist(fluor, 100)
    break
    
    
#     plot_gridded_lowess(y_pos, x_pos, fluor, yx_query_points, ax[0, c_index], N =20):

### Use normalized LOWESS to do shading correction

In [ ]:
#TODO
background_subtracted_lowess_names = 

for j, channel in enumerate(background_subtracted_lowess_names):            
    print(channel)
    # 1) Load background subtracted
    y_pos = bsccm.index_dataframe['position_in_fov_y_pix'].to_numpy()
    x_pos = bsccm.index_dataframe['position_in_fov_x_pix'].to_numpy()
    fluor = bsccm.surface_marker_dataframe[channel].to_numpy()

    #TODO:
    # 2a) Compute LOWESS on normalized data
    
    #                 yx_query_points = np.stack([y_pos, x_pos], axis=-1)

#                 # compute alpha adaptively so it corresponds to same number cells
#                 alpha = 2500 / x_pos.shape[0] 
#                 predictions = lowess_2d(y_pos, x_pos, fluor, 
#                           yx_query_points, alpha=alpha, weight_fn='tricubic')
    
    
    # 2b) Divide by shading correction factor
    
    
    
    # 3) Store result
    
    bsccm.surface_marker_dataframe.loc[mask_indices, channel_names[j][:-3] + '_shading_corrected'] = predictions

#Resave
home = str(Path.home())
data_root = home + '/BSCCM_local/BSCCM/'

### Validate the final result by looking at spatial histograms of normalized fluorescence

In [ ]:
corrected_channel_names = [
    'Fluor_426-446_total_shading_corrected',
    'Fluor_500-550_total_shading_corrected',
    'Fluor_550-570_total_shading_corrected',
    'Fluor_585-625_total_shading_corrected',
    'Fluor_627-673_total_shading_corrected',
    'Fluor_690-_total_shading_corrected',
    ]

show_spatial_histograms(corrected_channel_names)